In [4]:
import os
import nbformat
import pkg_resources

def extract_imports_from_notebook(notebook_path, custom_libraries):
    with open(notebook_path, 'r', encoding='utf-8') as f:
        notebook = nbformat.read(f, as_version=4)
    
    imports = set()
    for cell in notebook.cells:
        if cell.cell_type == 'code':
            code = cell.source
            lines = code.split('\n')
            for line in lines:
                line = line.strip()
                if line.startswith('import ') or line.startswith('from '):
                    parts = line.replace(',', ' ').split()
                    if parts[0] == 'import':
                        for part in parts[1:]:
                            lib = part.split('.')[0]
                            if lib and lib not in custom_libraries:
                                imports.add(lib)
                    elif parts[0] == 'from':
                        lib = parts[1].split('.')[0]
                        if lib and lib not in custom_libraries:
                            imports.add(lib)
    return imports

def get_installed_version(lib_name):
    try:
        version = pkg_resources.get_distribution(lib_name).version
    except pkg_resources.DistributionNotFound:
        version = None
    return version

def generate_requirements(custom_libraries=None):
    if custom_libraries is None:
        custom_libraries = []
    
    current_directory = os.getcwd()
    all_imports = set()
    
    for root, _, files in os.walk(current_directory):
        for file in files:
            if file.endswith('.ipynb'):
                notebook_path = os.path.join(root, file)
                imports = extract_imports_from_notebook(notebook_path, custom_libraries)
                all_imports.update(imports)
    
    requirements = []
    for lib in sorted(all_imports):
        version = get_installed_version(lib)
        if version:
            requirements.append(f'{lib}=={version}')
    
    with open('requirements.txt', 'w') as f:
        for req in requirements:
            f.write(req + '\n')

    print(f"requirements.txt generated with {len(requirements)} libraries.")

# List your custom libraries here
custom_libraries = ['model', 'your_custom_library']
generate_requirements(custom_libraries)


requirements.txt generated with 24 libraries.


In [5]:
!pip install py-cpuinfo psutil gputil


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7418 sha256=12314f09b9af1577b47c1df3a537502a0da42003be2bfeddcf29e6c98cfaa8e9
  Stored in directory: c:\users\edayo\appdata\local\pip\cache\wheels\a9\8a\bd\81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import cpuinfo
import psutil
import GPUtil

def get_important_system_info():
    # CPU Info
    cpu_info = cpuinfo.get_cpu_info()
    important_cpu_info = {
        'brand': cpu_info.get('brand_raw'),
        'arch': cpu_info.get('arch'),
        'count': cpu_info.get('count'),
        'hz_advertised': cpu_info.get('hz_advertised_friendly')
    }

    # Memory Info
    memory_info = psutil.virtual_memory()
    important_memory_info = {
        'total': memory_info.total,
        'available': memory_info.available,
        'used': memory_info.used,
        'percent': memory_info.percent
    }

    # Disk Info
    disk_info = psutil.disk_usage('/')
    important_disk_info = {
        'total': disk_info.total,
        'used': disk_info.used,
        'free': disk_info.free,
        'percent': disk_info.percent,
        'meets_minimum_storage': disk_info.free >= 5 * 1024 * 1024 * 1024  # 5GB in bytes
    }

    # Combine all important info
    important_system_info = {
        'cpu_info': important_cpu_info,
        'memory_info': important_memory_info,
        'disk_info': important_disk_info
    }

    return important_system_info

info = get_important_system_info()
print(info)


{'cpu_info': {'brand': '11th Gen Intel(R) Core(TM) i5-11320H @ 3.20GHz', 'arch': 'X86_64', 'count': 8, 'hz_advertised': '3.2000 GHz'}, 'memory_info': {'total': 16905965568, 'available': 6061633536, 'used': 10844332032, 'percent': 64.1}, 'disk_info': {'total': 486354251776, 'used': 353266790400, 'free': 133087461376, 'percent': 72.6, 'meets_minimum_storage': True}}
